In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from pandas.io import sql
from sqlalchemy import create_engine, inspect
from nltk.tokenize import word_tokenize          
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

## Get the data we want to analyze

In [27]:
files = ['/data/reddit1.csv', 
         '/data/reddit2.csv',
         '/data/reddit3.csv',
         '/data/reddit4.csv',
         '/data/reddit5.csv',
         '/data/reddit6.csv',
         '/data/reddit7.csv',
         '/data/reddit8.csv',
         '/data/reddit9.csv',
         '/data/reddit10.csv']

finalDf = pd.DataFrame()

for file in files:
    df = pd.read_csv("/data/reddit1.csv")
    df = df[['body', 'created_utc', 'controversiality']]
    #adding the controversial 
    controversial = df[df['controversiality'] == 1].copy().reset_index()
    del controversial['index']
    finalDf = finalDf.append(controversial,ignore_index = True)
    #adding the non-controversial 
    non_controversial = df[df['controversiality'] == 0].sample(frac=0.1, replace=False).copy().reset_index()
    del non_controversial['index']
    finalDf = finalDf.append(non_controversial, ignore_index = True)

df = finalDf

# Need to create columns for 
    1. Time of Day
    2. Day of Week
    3. Word Count
    4. top 500 word for controversial (tfidf value)
    5. top 500 word for non-controversial (tfidf value)

In [28]:
df['body'] = df['body'].astype('str')
df['length']=df['body'].str.split(' ').str.len()

### Construct contraversial word list

In [71]:
top_n_words = 500

In [33]:
controversial = df[df['controversiality'] == 1]
comments = controversial['body']
vectorizer = TfidfVectorizer(min_df=1, stop_words='english', analyzer='word')
X = vectorizer.fit_transform(comments)
idf = vectorizer.idf_

tfidf_score = dict(zip(vectorizer.get_feature_names(), idf))
tfidf_score_sorted = sorted(tfidf_score.items(), key=operator.itemgetter(1), reverse=True)

controversial_words = list( tfidf_score_sorted[:top_n_words][x][0] for x in range(top_n_words))

In [34]:
non_controversial = df[df['controversiality'] == 0]
comments = non_controversial['body']
vectorizer = TfidfVectorizer(min_df=1, stop_words='english', analyzer='word')
X = vectorizer.fit_transform(comments)
idf = vectorizer.idf_

tfidf_score = dict(zip(vectorizer.get_feature_names(), idf))
tfidf_score_sorted = sorted(tfidf_score.items(), key=operator.itemgetter(1), reverse=True)

non_controversial_words = list( tfidf_score_sorted[:top_n_words][x][0] for x in range(top_n_words))

In [35]:
words=[]
for word in non_controversial_words: 
    df[word+'count']=0
    words.append(word)
for word in controversial_words:
    df[word+'count']=0
    words.append(word)

In [38]:
# dict for easy lookup
ws = {}
for w in words:
    ws[w] = True

import re
d = {}
for row in df.itertuples():
    for word in re.findall(r"[\w']+", row[1].strip().lower()):
        if word in ws:
            if (word+'count', row[0]) not in d:
                d[(word+'count', row[0])] = 0
            d[(word+'count', row[0])] += 1    
            
# add the dict to df            
for (w, i) in d:
    df.set_value(i, w, d[(w, i)])

del d

In [40]:
# Verify that sum > 0
num = 0
for col in df.columns:
    if col != 'body':
        if sum(df[col]) == 0:
            num += 1
# TODO: Delete cols that have sum 0            
num

156

In [15]:
df.head(5)

,body,created_utc,controversiality,length,つーかリンク先読んだらナチュラルにテンプレ通りの政治スレ禁止厨でワロタcount,starlogcount,mincomecount,母乳って元々血液なんだっけcount,bulletingcount,inexorablycount,...,audiophilecount,neurotoxincount,derisioncount,cidcount,sarcasitccount,pansiecount,absurditycount,impressivenesscount,skynetcount,fighter_imagescount
0,Because we aren't responsible for the actions ...,1430438401,1,328,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,I honestly wouldn't have believed it if I didn...,1430438402,1,38,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,There is also many intelligence service player...,1430438402,1,32,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The implications of that varies between cultur...,1430438405,1,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,*I am a bot whose sole purpose is to improve t...,1430438406,1,297,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import numpy as np 
from sklearn.linear_model import LassoCV, lasso_path
from sklearn.cross_validation import cross_val_score 
lasso_fit = LassoCV(cv = 10, n_alphas=50, max_iter=100000, normalize=True)
dfminusresponse=df.drop('controversiality',axis=1)
dfminusresponse=dfminusresponse.drop('body',axis=1)
lasso_fit.fit(dfminusresponse, df['controversiality'])
lasso_fit.coef_

In [ ]:
i=0
j=0
for element in lasso_fit.coef_:
    if element>0:
        i+=1
    else:
        j+=1
print(i,j)


In [ ]:
df

In [ ]:
from sklearn.linear_model import ElasticNet
elastic_fit=ElasticNet(alpha=50,l1_ratio=.9,max_iter=100000,normalize=True)
elastic_fit.fit(df.drop('controversiality',axis=1).drop('body',axis=1),df['controversiality'])

In [ ]:
i=0
for value in lasso_fit.coef_:
    if value==0:
        
    i+=1

In [ ]:
i=1
col=[]
for coef in lasso_fit.coef_:
    if coef==0:
        col.append(i)
    i+=1
df=df.drop('prediction',axis=1)

In [ ]:
new_df=df.drop('body',axis=1).drop('controversiality',axis=1).drop(df.columns[col],axis=1)


In [ ]:
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(new_df,df['controversiality'])


In [ ]:
print(len(df.ix[df['controversiality']==0])/len(df))
print(model1.score(new_df,df['controversiality']))

## Ignore

In [ ]:
from nltk.tokenize import word_tokenize          
import operator

df['body'] = df['body'].astype('str')

comments = df['body']

word_count={}
punctuations = '''`!()-[]{};:'"\,<>./?@#$%^&*_~=+ '''

for comment in comments:
    for word in word_tokenize(comment.lower()):
        if word not in punctuations:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

sorted_x = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)